In [35]:
import time, re, pdfx, unidecode
import datetime

# hyper-parameters
ref_author = True
min_number_of_common_works = 6
number_of_top = 10

# laod files into RDD
csv_articles = sc.textFile("data/export_articles_EGC_2004_2018.csv")
dico_fr = sc.textFile("dictionnary/lexique-collecte.txt")
dico_en = sc.textFile("dictionnary/words.txt")
stop_word_fr = sc.textFile("dictionnary/stop-word-french.txt")
stop_word_en = sc.textFile("dictionnary/stop-word-en.txt")

# print the date of beginning
date1 = datetime.datetime.now()
print("Date Begin", date1)

Date Begin 2020-01-23 00:48:47.805852


In [36]:
# split data for csv file
def split_line_articles(line):   # create list of article as a tuple
    items = line.split("\t")
    return (items[0].lower(), items[1].lower(), items[2].lower(), items[3].lower(), items[4].lower(), items[5].lower(), items[6].lower())

def split_line_dico_fr(line):
    items = line.split("\t")
    return items[2].lower()

csv_articles = csv_articles.map(split_line_articles)
dico_fr = dico_fr.map(split_line_dico_fr)
dico_en = dico_en.map(lambda w : w.lower())
stop_word_fr = stop_word_fr.map(lambda w : w.lower())
stop_word_en = stop_word_en.map(lambda w : w.lower())

print(csv_articles.collect()[1:2])
print(dico_fr.collect()[:10])
print(dico_en.collect()[:20])
print(stop_word_fr.collect()[:30])
print(stop_word_en.collect()[:30])

[("revue des nouvelles technologies de l'information", 'egc', '2018', "#idéo2017 : une plateforme citoyenne dédiée à l'analyse des tweets lors des événements politiques", "cette plateforme a pour objectif de permettre aux citoyens d'analyserpar eux-mêmes les tweets politiques lors d'événements spécifiques en france.pour le cas de l'élection présidentielle de 2017, #idéo2017 analysait en quasitemps réel les messages des candidats, et fournissait leurs principales caractéristiques,l'usage du lexique politique et des comparaisons entre les candidats.", 'claudia marinica, julien longhi, nader hassine, abdulhafiz alkhouli, boris borzic', 'http://editions-rnti.fr/render_pdf.php?p1&p=1002425')]
['flexion', 'de', 'de', 'et', 'à', 'des', 'du', 'que', 'la', 'la']
['2', '1080', '&c', '10-point', '10th', '11-point', '12-point', '16-point', '18-point', '1st', '2,4,5-t', '2,4-d', '20-point', '2d', '2nd', '30-30', '3d', '3-d', '3m', '3rd']
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 

In [37]:
# find the name of authors in a string
def find_ref(refs):
    result = []
    groups = re.search(r'[A-Z][a-z\-]*\s*,\s*[A-Z]|(,\s*|(et|and)\s*)[A-Z]\s*[A-Z]\s*[A-Za-zç\-\']*', refs)

    if groups:
        value = groups.group()
        # process the beginning of name
        if value.startswith(",et"):
            value = value[3:]
        elif value.startswith(", et"):
            value = value[4:]
        elif value.startswith(" et"):
            value = value[3:]
        elif value.startswith("et"):
            value = value[2:]
        elif value.startswith(",and"):
            value = value[4:]
        elif value.startswith(", and"):
            value = value[5:]
        elif value.startswith(" and"):
            value = value[4:]
        elif value.startswith("and"):
            value = value[3:]
        elif value.startswith(","):
            value = value[1:]

        if value.startswith(" "):
            value = value[1:]
        elif value.startswith("  "):
            value = value[2:]

        # change the form to get always the same format of name
        if re.match(r'[A-Za-z\-]*\s*,\s*[A-Z]', value):
            value = value[value.find(",") + 1:] + " " + value[:value.find(",")]

        # process the spaces at the beginning of name
        if value.startswith(" "):
            value = value[1:]
        elif value.startswith("  "):
            value = value[2:]

        # process the spaces at the end of name
        if value.endswith(" "):
            value = value[:-1]
        elif value.endswith("  "):
            value = value[:-2]

        # convert accent characters
        #value = unidecode.unidecode(value)

        # last verification to remove common words
        if re.match(r'[A-Z](\s|-)', value):
            result.append(value)

        return result + find_ref(refs[groups.span()[1]:])

    return result

In [38]:
# load all text of pdf
def load_text_pdf(url, id):
    refs = ""
    text = ""
    marker = -1
    reference_find = False
    
    try:
        pdf = pdfx.PDFx("data/pdf/" + id + ".pdf")
        text = pdf.get_text()
    except: print("Failed", id)
    
    if text != "":
        pages = text.split("\n\n")
        for p in range(len(pages) - 1, 0, -1):   # we search on each page p from the end ...
            for s in pages[p].split("."):   # ... on each sentence s ...
                if "References" in s or "Références" in s:  # ... the page with references part
                    marker = p
                    break
            if marker != -1:
                break

        if marker != -1:
            for p in range(marker, len(pages)):   # we process only the pages with references
                for s in pages[p].split("."):
                    if reference_find or "References" in s or "Références" in s:  # ... the page with references part
                        refs += s.replace("\n", "") + " "  # we keep all characters after the references have benn found
                        reference_find = True
    return refs

def get_text_pdf(art):
    url = art[6]
    id = url[url.find("&") + 3:]
    
    begin = time.time()
    contentPdf = load_text_pdf(url, id)
    
    return (id, contentPdf, (time.time() - begin))

In [39]:
# get the references part of the article
def parse_references_aux(text):
    refs = ""
    marker = -1
    reference_find = False
    
    pages = text.split("\n\n")
    for p in range(len(pages) - 1, 0, -1):   # we search on each page p from the end ...
        for s in pages[p].split("."):   # ... on each sentence s ...
            if "References" in s or "Références" in s:  # ... the page with references part
                marker = p
                break
        if marker != -1:
            break

    if marker != -1:
        for p in range(marker, len(pages)):   # we process only the pages with references
            for s in pages[p].split("."):
                if reference_find or "References" in s or "Références" in s:  # ... the page with references part
                    refs += s.replace("\n", "") + " "  # we keep all characters after the references have benn found
                    reference_find = True
    
    return refs
    
def parse_references(art):
    
    begin = time.time()
    contentPdf = parse_references_aux(art[1])
    
    return (art[0], contentPdf, art[2] + (time.time() - begin))

In [40]:
# the name of authors for each quotation
def get_names(refs):
    content_pdf = []
    
    if refs != "":  # if we have references in this article
        for r in re.split(r'\([1-2][0-9][0-9][0-9]\)', refs):  # we split per date
            content_pdf.append(find_ref(unidecode.unidecode(r)))   # we search the name of authors

    return content_pdf
    
def get_names_of_references(art):
    
    begin = time.time()
    contentPdf = get_names(art[1])
    
    return (art[0], contentPdf, art[2] + (time.time() - begin))

In [41]:
csv_articles = csv_articles.map(get_text_pdf)
articles = csv_articles.collect()
#print("GET PDF TEXT")
print("Articles : ", len(articles[1][1]), articles[1][2])

Articles :  1546 0.2553408145904541


In [42]:
#csv_articles = csv_articles.map(parse_references)
#articles = csv_articles.collect()
#print("GET PDF REFS")
#for art in articles[1:2]:
#    print("Articles : ", art[1], art[2])

In [43]:
csv_articles = csv_articles.map(get_names_of_references)
articles = csv_articles.collect()
print("GET AUTHORS")
for art in articles[1:10]:
    print("Articles : ", art[1], art[2])

GET AUTHORS
Articles :  [['C Alduy'], ['M Conover', 'J Ratkiewicz', 'M Francisco', 'B Goncalves', 'A Flammini', 'F Menczer'], ['D Avello', 'P T Metaxas', 'E Mustafaraj'], ['R Adamic', 'S Counts', 'J Longhi'], ['J Longhi', 'C Marinica', 'N Haddioui'], ['C Negationetreference', 'A Mercier'], ['S Roginsky', 'B D Cock'], ['A Tumasjan', 'T O Sprenger', 'P G Sandner', 'I M Welpe'], []] 0.2373671531677246
Articles :  [['J Bennett', 'S Lanning'], ['H Bock'], ['M Boulle'], ['G Guyon', 'G Cawley', 'A Saffari', 'C Castillo', 'K Chellapilla', 'L Denoyer'], ['C Beijing', 'I Dhillon', 'S S', 'D S Modha'], ['K Mining', 'N York', 'U Y', 'G Govaert', 'M Nadif'], ['G Govaert', 'M Nadif'], ['R Guigoures', 'D Gay', 'M Boulle', 'F Clerot', 'F Rossi'], ['J Hartigan'], ['H Li', 'N Abe'], ['P Stroudsburg', 'U A', 'I Mechelen', 'H V', 'P D Boeck'], ['T Mitchell'], ['N York', 'U Y', 'I Hill', 'S Papadimitriou', 'J Sun'], []] 0.7898545265197754
Articles :  [['R A', 'A J', 'C J Tauro'], ['M Ghossein', 'T Abdessal

In [44]:
def sum(a, b):
    #print(a + b)
    return a + b

csv_articles = csv_articles.map(lambda art : art[2])
t = csv_articles.reduce(sum)

# calculate and print execution time to process the articles only
print("Time to execute parsing", t)

1748.8845024108887
Time to execute parsing 1748.8845024108887


In [45]:
# print the date of end to know total execution time
date2 = datetime.datetime.now()
print("Date End", date2)

Date End 2020-01-23 01:34:26.307230
